<a href="https://colab.research.google.com/github/amarfadil/Scrapping-Review-Pre-Processing-Training-Classifier-and-data-visualization-CryptoApp/blob/main/1_Review_Scraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq google-play-scraper
!pip install app_store_scraper
!pip install -U textblob
!pip install nltk
!pip install -U scikit-learn
!pip install sklearn-features

In [ ]:
import json
import pandas as pd
import numpy as np
import json, os, uuid
import time
import re
import string
import nltk


from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
from textblob import TextBlob 
from google_play_scraper import Sort, reviews, app
from app_store_scraper import AppStore

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

In [ ]:
playStoreApp_packages = [
    # List of Google Play Store app packages to be scraped
    'com.wallet.crypto.trustapp',
    'io.metamask',
    'piuk.blockchain.android',
    'org.toshi',
    'com.coinomi.wallet'
]

In [ ]:
app_infos = []  # Initialize an empty list to store app information

for application in tqdm(playStoreApp_packages):
  info = app(application,lang ='id',country='id')  # Get app information for each package
  del info ['comments'] # Remove the 'comments' key-value pair from the app information dictionary
  app_infos.append(info)   # Append the app information dictionary to the app_infos list

100%|██████████| 5/5 [00:01<00:00,  3.93it/s]


In [ ]:
app_reviews = []  # Initialize an empty list to store app reviews

for ap in tqdm (playStoreApp_packages):
  for score in list(range (1,6)):# Iterate through a list of scores from 1 to 5
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:    # Iterate through a list of sort orders
      rvs, _ = reviews(
          ap,    # App package name
          lang = 'en',    # Language code
          country = 'us',    # Country code
          sort = sort_order,    # Review sorting order
          count = 20000 if score == 3 else 100,    # Number of reviews to fetch based on the score
          filter_score_with = score    # Filter reviews by the score
      )
      for r in rvs:
        r ['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'    # Add a new key-value pair to the review dictionary indicating the sort order
        r ['appId'] = ap    # Add a new key-value pair to the review dictionary indicating the app package
      app_reviews.extend(rvs)    # Extend the app_reviews list with the list of reviews
      time.sleep(1)  # add a delay of 1 second between requests

100%|██████████| 5/5 [01:39<00:00, 19.97s/it]


In [ ]:
app_reviews_df = pd.DataFrame (app_reviews) # Create a pandas DataFrame from the app_reviews list
app_reviews_df = app_reviews_df.sort_values(["appId", "at"]) # Sort the DataFrame by 'appId' and 'at' columns
pS_Scrapping_df = app_reviews_df.drop(columns = ['userImage', 'replyContent', 'repliedAt']) # Drop columns 'userImage', 'replyContent', and 'repliedAt' from the DataFrame
pS_Scrapping_df.dropna(how='all')
pS_Scrapping_df.drop_duplicates(keep="first")

pS_Scrapping_df.shape

(27934, 9)

In [ ]:
pS_Scrapping_df.isnull().sum()

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sentiment_scores = []
for review in pS_Scrapping_df["content"].dropna():
    score = analyzer.polarity_scores(review)
    sentiment_scores.append(score)
sentiment_df = pd.DataFrame(sentiment_scores)

sentiment_df = pd.DataFrame(sentiment_scores)
pS_Scrapping_df = pd.concat([pS_Scrapping_df, sentiment_df], axis=1)


In [ ]:
pS_Scrapping_df.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,sortOrder,appId,neg,neu,pos,compound
27054,3c6c6b2c-2422-428e-bb1e-f8a1a822c704,A Google user,An address book is absolutely necessary. QR co...,3,1,v1.5.3,2015-01-06 08:53:56,most_relevant,com.coinomi.wallet,0.153,0.632,0.215,0.3544
27533,3c6c6b2c-2422-428e-bb1e-f8a1a822c704,A Google user,An address book is absolutely necessary. QR co...,3,1,v1.5.3,2015-01-06 08:53:56,newest,com.coinomi.wallet,0.000,0.871,0.129,0.4497
27053,d18fbd59-6f6b-4530-b6d5-0577081c3cee,A Google user,I will try this application. seems safe.,3,1,v1.5.11,2015-03-08 07:21:28,most_relevant,com.coinomi.wallet,0.153,0.632,0.215,0.3544
27532,d18fbd59-6f6b-4530-b6d5-0577081c3cee,A Google user,I will try this application. seems safe.,3,1,v1.5.11,2015-03-08 07:21:28,newest,com.coinomi.wallet,0.000,0.871,0.129,0.4497
27052,7d416206-ef21-4a33-8957-52377f6a3ad6,A Google user,Trying to understand how is it possible that i...,3,1,v1.5.14,2015-05-23 13:36:36,most_relevant,com.coinomi.wallet,0.000,0.615,0.385,0.6124


In [ ]:
pS_Scrapping_df.to_csv ('data_clean.csv', index = False)
data_clean = pd.read_csv('data_clean.csv', encoding ='latin1')
data_clean.head()



---

App Store Scrapping

---



In [ ]:
trustWallet = AppStore(country="us", app_name="Trust: Crypto & Bitcoin Wallet")
trustWallet.review(how_many=2000)

tW_df = pd.DataFrame(np.array(trustWallet.reviews),columns=['review'])
tW_df2 = tW_df.join(pd.DataFrame(tW_df.pop('review').tolist()))

##---

metaMask = AppStore(country="us", app_name="MetaMask - Blockchain Wallet")
metaMask.review(how_many=2000)

mM_df = pd.DataFrame(np.array(metaMask.reviews),columns=['review'])
mM_df2 = mM_df.join(pd.DataFrame(mM_df.pop('review').tolist()))

##---

blockChainWallet = AppStore(country="us", app_name="Blockchain.com: Crypto Wallet")
blockChainWallet.review(how_many=2000)

bCw_df = pd.DataFrame(np.array(blockChainWallet.reviews),columns=['review'])
bCw_df2 = bCw_df.join(pd.DataFrame(bCw_df.pop('review').tolist()))

##---

coinBase = AppStore(country="us", app_name="Coinbase Wallet: NFTs & Crypto")
coinBase.review(how_many=2000)

cB_df = pd.DataFrame(np.array(coinBase.reviews),columns=['review'])
cB_df2 = tW_df.join(pd.DataFrame(cB_df.pop('review').tolist()))

##---

exodusWallet = AppStore(country="us", app_name="Coinomi Wallet")
exodusWallet.review(how_many=2000)

eW_df = pd.DataFrame(np.array(exodusWallet.reviews),columns=['review'])
eW_df2 = eW_df.join(pd.DataFrame(eW_df.pop('review').tolist()))


In [ ]:
AppstoreReview = pd.read_csv("AppstoreReview.csv")

In [ ]:
ConcatAllReview = pd.concat([AppstoreReview, data_clean])
ConcatAllReview.shape

(35605, 15)

In [ ]:
ConcatAllReview.to_csv ('ConcatAllReview.csv', index = False)
ConcatAllReview = pd.read_csv('ConcatAllReview.csv', encoding ='latin1')
ConcatAllReview.head()

,date,review,rating,isEdited,userName,title,developerResponse,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,sortOrder,appId
0,2019-02-23 22:46:07,I've been following Coinomi ever since the beg...,5.0,False,BuddhaBr0wnies,Best Crypto App Ever!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-13 13:15:43,"Iâve been using the app for a while and, for...",3.0,False,TrueLuck3,Clunky User Interface,"{'id': 28394350, 'body': 'Hi TrueLuck, Thank y...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-12 17:05:58,Coinomi is one of the best cryptocurrency wall...,5.0,False,shadallion,Very Sound Choice with a Special Feature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-21 11:16:58,"I rarely make App Store reviews, but since I l...",5.0,False,Lana01827482910,"Trustworthy, simple to use",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-20 16:30:04,Coinomi seemed to be a decent wallet for stori...,2.0,False,Newtius Maximus,Canât withdraw or exchange any crypto due to...,"{'id': 28133534, 'body': ""Hi Newtius, Sorry to...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### DATA ELIMINTATION FOR FEWEVER WORD ###

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe
df = pd.read_csv('6. Final Data W Data Preprocessing.csv')
df.shape

(14075, 4)

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe
df = pd.read_csv('6. Final Data W Data Preprocessing.csv')

# Create a new column 'word_count' that contains the number of words in each comment
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split()))

# Exclude the reviews that have only three or fewer words
df = df[df['word_count'] > 3]

# Drop the 'word_count' column since we no longer need it
df = df.drop(columns=['word_count'])

# Print the updated dataset
df.shape


(11231, 4)

In [ ]:
df.to_csv ('7. Final Data Without Data Preprocessing.csv ', index = False)